In [2]:
from urllib.request import urlopen
import json
import os

import numpy as np
import pandas as pd

import csv
# import plotly.express as px
# import plotly.graph_objects as go  # or plotly.express as px

# from .about_us import about_us_html

In [3]:
def county2fips_fct(x):
    if x in county2fips:
        return str(int(county2fips[x]))
    else:
        return np.nan


# # define data dir
data_dir = "../data"
data_filepath = data_dir
# data_filepath = os.path.join(os.path.dirname(os.path.realpath(__file__)), data_dir)

counties_with_latlong = pd.read_csv(
    os.path.join(data_filepath, "counties.txt"), dtype={"GEOID": str}, sep="\t"
)

# get sundown town data
df = pd.read_csv(os.path.join(data_filepath, "sundown_with_counties.csv"), encoding="latin-1")
df["County_no_states"] = df.county.str.split(",").apply(lambda x: x[0])
county_long_names = []
for i in df.county.values:
    x = i.split(",")
    if len(x) == 2:
        county_long_names.append(x[0] + " " + x[1])
    else:
        county_long_names.append(x)
df["county_long_names"] = county_long_names

# get fips data
fips_codes = pd.read_csv(os.path.join(data_filepath, "county_fips_master.csv"), encoding="latin-1")
county2fips = dict(fips_codes[["county_name", "fips"]].values)

In [4]:
county_sundown_counts = pd.DataFrame(df.groupby(by="County_no_states").size())
county_sundown_counts = county_sundown_counts.reset_index()
county_sundown_counts = county_sundown_counts.rename(columns={"County_no_states": "County", 0: "#"})
county_sundown_counts["fips"] = county_sundown_counts.County.apply(lambda x: county2fips_fct(x))
county_sundown_counts = county_sundown_counts[county_sundown_counts.fips.notnull()]

with urlopen(
    "https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json"
) as response:
    counties = json.load(response)

df = pd.read_csv(
    "https://raw.githubusercontent.com/plotly/datasets/master/fips-unemp-16.csv",
    dtype={"fips": str},
)

tab_style = {
    "borderBottom": "1px solid #d6d6d6",
    "padding": "6px",
    "margin-bottom": "10px",
    "fontWeight": "bold",
}

tab_selected_style = {
    "borderTop": "1px solid #d6d6d6",
    "borderBottom": "1px solid #d6d6d6",
    "padding": "6px",
    "margin-bottom": "10px",
}

In [5]:
counties_with_latlong = pd.read_csv('../data/counties.txt', dtype={"GEOID": str}, sep='\t', skipinitialspace=True)

In [6]:
counties_with_latlong.USPS.unique()

array(['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA',
       'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA',
       'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY',
       'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
       'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY', 'PR'], dtype=object)

In [7]:
city_county_dict = dict()
with open('../data/us_cities_to_county.csv') as city_mapping_file:
    city_mapping_reader = csv.DictReader(city_mapping_file)
    for c in city_mapping_reader:
        # need to add in state, otherwise get missed mappings
        city_county_dict[(c['city'] + ' ' + c['state_id']).lower()] = c['county_name']

In [8]:
import pandas as pd

sundown_data = {}
with open('../data/sundown_states.csv') as sundown_file:
    city_reader = csv.DictReader(sundown_file)
    for r in city_reader:
        city_state = r['city'] + ' ' + r['state']
        if city_state.lower() in city_county_dict.keys():
            data = r
            data['county'] = city_county_dict[city_state.lower()] + ' County' +', ' + r['state']
            sundown_data[r['id']] = data
        elif 'County' in r['city']:
            print('missed mapping: {}'.format(r['city']))
            data['county'] = r['city']
            sundown_data[r['id']] = data



missed mapping: Buchanan County
missed mapping: Pulaski County
missed mapping: Taylor County
missed mapping: Clay County
missed mapping: Newton County
missed mapping: Cleburne County
missed mapping: Scott County
missed mapping: Stone County
missed mapping: Spink County
missed mapping: Orange County
missed mapping: Wheeler County
missed mapping: Comanche County
missed mapping: Hamilton County
missed mapping: Montague County
missed mapping: Marion County
missed mapping: Del Norte County
missed mapping: Cullman County
missed mapping: Antelope County
missed mapping: Burt County
missed mapping: Butler County
missed mapping: Cass County
missed mapping: Cedar County
missed mapping: Cheyenne County
missed mapping: Humboldt County
missed mapping: Dallam County
missed mapping: Scurry County
missed mapping: Mitchell County
missed mapping: Graham County
missed mapping: Forsyth County
missed mapping: Menominee County
missed mapping: Greer County
missed mapping: Ottawa County
missed mapping: Swain C

In [9]:
sundown_with_counties_df = pd.DataFrame(sundown_data.values())
sundown_with_counties_df.to_csv('../data/sundown_with_counties.csv')

In [10]:
sundown_with_counties_df

,id,city,state,status,county
0,8,Anna,IL,Surely,"Union County, IL"
1,11,Mize,MS,Surely,"Smith County, MS"
2,13,d'Iberville,MS,Possible,"Harrison County, MS"
3,14,Clinton,MS,Unlikely,"Hinds County, MS"
4,15,Pearl,MS,Possible,"Rankin County, MS"
5,16,Southaven,MS,Possible,"DeSoto County, MS"
6,17,Burnsville,MS,Probable,"Tishomingo County, MS"
7,18,Belmont,MS,Possible,Buchanan County
8,18,Belmont,MS,Possible,Buchanan County
9,20,Colonial Heights,VA,Probable,"Colonial Heights County, VA"


In [11]:
# unique values of sundown towns
sundown_with_counties_df['status'].unique()

array(['Surely', 'Possible', 'Unlikely', 'Probable',
       'Black Town or Township', "Don't Know", 'Always Biracial', ''],
      dtype=object)

In [12]:
counties_with_latlong.columns = map(lambda s:s.strip(), counties_with_latlong.columns)

In [13]:
matching_state = counties_with_latlong[counties_with_latlong["USPS"] == 'AL']

In [14]:
matching_state.INTPTLONG

0    -86.646440
1    -87.746067
2    -85.405104
3    -87.127148
4    -86.566440
5    -85.717261
6    -86.681969
7    -85.827909
8    -85.394032
9    -85.654242
10   -86.726607
11   -88.248889
12   -87.818624
13   -85.863525
14   -85.516126
15   -85.989602
16   -87.801457
17   -86.988722
18   -86.243482
19   -86.448721
20   -86.319222
21   -86.869267
22   -85.609476
23   -87.114356
24   -85.803992
25   -86.142735
26   -87.168410
27   -86.034263
28   -87.764292
29   -87.842814
        ...    
37   -88.087431
38   -87.650997
39   -87.321865
40   -85.353048
41   -86.981399
42   -86.650586
43   -85.692887
44   -86.551080
45   -87.791091
46   -87.881551
47   -86.321668
48   -88.196568
49   -87.383266
50   -86.204461
51   -86.846402
52   -87.293827
53   -88.096863
54   -85.941604
55   -85.464068
56   -85.186980
57   -86.315663
58   -86.678089
59   -88.200057
60   -86.175930
61   -85.799618
62   -87.522783
63   -87.301094
64   -88.197564
65   -87.304935
66   -87.365346
Name: INTPTLONG, Length:

In [15]:
latlon = matching_state[matching_state.columns[-2:]].values.tolist()[0]

In [16]:
matching_state

,USPS,GEOID,ANSICODE,NAME,ALAND,AWATER,ALAND_SQMI,AWATER_SQMI,INTPTLAT,INTPTLONG
0,AL,01001,161526,Autauga County,1539602137,25706961,594.444,9.926,32.532237,-86.646440
1,AL,01003,161527,Baldwin County,4117621645,1132980868,1589.823,437.446,30.659218,-87.746067
2,AL,01005,161528,Barbour County,2292160139,50523213,885.008,19.507,31.870253,-85.405104
3,AL,01007,161529,Bibb County,1612167476,9602089,622.461,3.707,33.015893,-87.127148
4,AL,01009,161530,Blount County,1670103866,15015467,644.831,5.798,33.977358,-86.566440
5,AL,01011,161531,Bullock County,1613076054,6038093,622.812,2.331,32.101759,-85.717261
6,AL,01013,161532,Butler County,2012002531,2701198,776.839,1.043,31.751667,-86.681969
7,AL,01015,161533,Calhoun County,1569195268,16587192,605.870,6.404,33.770516,-85.827909
8,AL,01017,161534,Chambers County,1545085607,16971700,596.561,6.553,32.915504,-85.394032
9,AL,01019,161535,Cherokee County,1433623319,120308342,553.525,46.451,34.069515,-85.654242


In [17]:
matching_state.INTPTLONG

0    -86.646440
1    -87.746067
2    -85.405104
3    -87.127148
4    -86.566440
5    -85.717261
6    -86.681969
7    -85.827909
8    -85.394032
9    -85.654242
10   -86.726607
11   -88.248889
12   -87.818624
13   -85.863525
14   -85.516126
15   -85.989602
16   -87.801457
17   -86.988722
18   -86.243482
19   -86.448721
20   -86.319222
21   -86.869267
22   -85.609476
23   -87.114356
24   -85.803992
25   -86.142735
26   -87.168410
27   -86.034263
28   -87.764292
29   -87.842814
        ...    
37   -88.087431
38   -87.650997
39   -87.321865
40   -85.353048
41   -86.981399
42   -86.650586
43   -85.692887
44   -86.551080
45   -87.791091
46   -87.881551
47   -86.321668
48   -88.196568
49   -87.383266
50   -86.204461
51   -86.846402
52   -87.293827
53   -88.096863
54   -85.941604
55   -85.464068
56   -85.186980
57   -86.315663
58   -86.678089
59   -88.200057
60   -86.175930
61   -85.799618
62   -87.522783
63   -87.301094
64   -88.197564
65   -87.304935
66   -87.365346
Name: INTPTLONG, Length: